In [ ]:
import datetime as dt
import pandas as pd
import requests
from tsfresh import extract_features, select_features 
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh.utilities.dataframe_functions import impute
import xgboost as xgb
import numpy as np
from sklearn.metrics import root_mean_squared_error
import matplotlib
import dask.dataframe as dd
from dask.dataframe import from_pandas
from dask.distributed import progress
from dask.diagnostics import ProgressBar

In [ ]:
from dask.distributed import Client, LocalCluster
import os
# Create a new directory for Dask's scraFONh space
os.makedirs('dask-scraFONh', exist_ok=True)
# Use the new directory as the local directory
cluster = LocalCluster(n_workers=20, local_directory='dask-scraFONh')
client = Client(cluster)
# client.scheduler.address

In [ ]:
import torch
# Check if CUDA is available
if torch.cuda.is_available():
    # Get the current device
    current_device = torch.cuda.current_device()
    # Get the name of the current device
    device_name = torch.cuda.get_device_name(current_device)
    print(f"CUDA is using: {device_name}")
else:
    print("CUDA is not available.")


In [ ]:
def get_proj_ids(coins):
    z = get_project_url_df()
    list_id = []
    for coin in coins:
        list_id.append(z.loc[z['symbol']== coin,'project_id'].item())
    return list_id
def get_project_url_df():
    url = "https://api.tokenterminal.com/v2/projects"
    headers = {
    "accept": "application/json",
    "authorization": TOKENTERMINAL_API_KEY
    }
    response = requests.get(url, headers=headers)
    list_of_dicts= eval(response.text.replace('null', 'None'))
    return pd.DataFrame(list_of_dicts['data'])
def get_specific_urls(coins):
    spec_urls = []
    for coin in coins:
        Boolean= get_project_url_df()['symbol'].eq(coin)
        count = 0
        for boo in Boolean: 
            if boo is True: 
                spec_urls.append(get_project_url_df().loc[count].url)
            else: count = count + 1 
    return spec_urls

In [ ]:
# import multiprocessing

# num_cores = multiprocessing.cpu_count()
# print(f"You have {num_cores} cores.")

In [ ]:
rolled

In [ ]:
def get_df_of_OCA(coins, date):
    list_dd = []
    headers = {
        "accept": "application/json",
        "authorization": TOKENTERMINAL_API_KEY
        }
    for url in get_specific_urls(coins):
        response = requests.get(f'https://api.tokenterminal.com{url}/metrics', headers=headers)
        data = response.text.replace("null", "None")
        raw_data_string = data[response.text.find("['")+9:response.text.find("']")]
        try: raw_data = eval(raw_data_string)
        except: continue
        list_dd.append(pd.DataFrame(raw_data))
    FON = list_dd[0]
    count = 0
    for list in list_dd:
        if count == 0: 
            count = count +1
            continue
        else:
            FON= pd.merge(left = FON, right= list_dd[count], how = 'outer', suffixes = (None, coins[count]))
            count = count + 1
            continue
    FON.drop(['project_name'], axis=1, inplace = True)
    FON['timestamp']=FON['timestamp'].apply(pd.to_datetime)
    FON= FON[(FON['timestamp'] >= date)]
    return dd.from_pandas(FON, npartitions=1)

In [ ]:
def roll(FON, timeshift):
    FON_rolled = roll_time_series(FON.compute(), column_id='project_id', column_sort='timestamp', max_timeshift=timeshift).drop('project_id', axis=1)
    for column in FON_rolled:
        if FON_rolled[column].dtypes =='float64' or  FON_rolled[column].dtypes == 'int64': 
            FON_rolled[column] = FON_rolled[column].fillna(FON_rolled[column].mean())
    # return dd.from_pandas(FON_rolled, npartitions=20)
    return FON_rolled
def extract_features_on_df(df):
    df = df.dropna(subset=['id', 'timestamp'])
    df = extract_features(df, column_id='id', column_sort='timestamp', n_jobs=1)
    return df


In [ ]:
# def roll(TC):
#     FON_rolled = roll_time_series(TC.compute(), column_id='project_id', column_sort='timestamp', max_timeshift=timeshift).drop('project_id', axis=1)
#     for column in FON_rolled:
#         if FON_rolled[column].dtypes =='float64' or  FON_rolled[column].dtypes == 'int64': 
#             FON_rolled[column] = FON_rolled[column].fillna(FON_rolled[column].mean())
#     return dd.from_pandas(FON_rolled, npartitions=10)
# def extract_features_on_df(df):
#     df = df.dropna(subset=['id', 'timestamp'])
#     df = extract_features(df, column_id='id', column_sort='timestamp', n_jobs=1)
# #     return df
# def select_features_inhouse(TC, FC, days_forecast):    
#     Relevant_Features_Target = {}
#     for proj_id in TC['project_id'].unique():
#         Y = TC[TC['project_id']==proj_id][['price', 'timestamp']].set_index('timestamp')
#         Y.sort_index(inplace=True)
#         Y['price'] = Y['price'].shift(-days_forecast)
#         Y['price'] = np.log(Y)-np.log(Y).shift(days_forecast)
#         Y.dropna(inplace=True)
#         X= FC.loc[proj_id].loc[Y.index].merge(FON[FON['project_id']==proj_id].set_index('timestamp').drop('project_id', axis =1), how = 'left', on = 'timestamp')
#         X['price'] = np.log(X['price'])-np.log(X['price'].shift(days_forecast))
#         if (proj_id != 'bitcoin'): 
#             X= X.merge(FON[FON['project_id']=='bitcoin'][['price','timestamp']].set_index('timestamp'), how = 'left', on = 'timestamp', suffixes=(None, '_bitcoin'))
#         X= impute(X)
#         Relevant_Features_Target[proj_id]= select_features(X, Y['price'])
#     return Relevant_Features_Target

# # def add_base_features(Selected, FON):
# #     for proj_id in FON['project_id'].unique():
# #         Selected[proj_id]= Selected[proj_id].merge(FON[FON['project_id']==proj_id].set_index('timestamp').drop('project_id', axis =1), how = 'left', on = 'timestamp')
# #         if (proj_id != 'bitcoin'): 
# #             Selected[proj_id]= Selected[proj_id].merge(FON[FON['project_id']=='bitcoin'][['price','timestamp']].set_index('timestamp'), how = 'left', on = 'timestamp', suffixes=(None, '_bitcoin'))
# #         else:
# #             continue
# #     return Selected

# # def add_base_features(Selected, FON):
# #     base_prices = {}
# #     base_volatility = {}
# #     for proj_id in FON['project_id'].unique():
# #         base_prices[proj_id] = FON[FON['project_id']==proj_id][['price', 'timestamp']].set_index('timestamp')
# #         base_prices[proj_id].rename(columns ={'price': (proj_id+ ': price')},inplace=True)
# #         base_volatility[proj_id] = base_prices[proj_id].ewm(alpha = 0.99).std().dropna()
# #         base_volatility[proj_id].rename(columns ={(proj_id+ ': price') : (proj_id+ ': volatility')},inplace=True)
# #     for proj_id in FON['project_id'].unique():
# #         Selected[proj_id]= Selected[proj_id].merge(base_prices[proj_id][proj_id +': price'], how = 'left', on = 'timestamp').merge(base_volatility[proj_id][proj_id +': volatility'] , how = 'left', on = 'timestamp')
# #         if (proj_id != 'bitcoin'): 
# #             Selected[proj_id]= Selected[proj_id].merge(base_prices['bitcoin']['bitcoin' +': price'], how = 'left', on = 'timestamp')
# #         else:continue
# #     return Selected

def train_xgbmodels_on_features(Xs,Y, proj_ids, days_forecast):
    for proj_id in proj_ids:
        X=  Xs.loc[proj_id]
        Y= Y.loc[proj_id]
        Y_train= Y.iloc[ : int(0.8*len(Y))]
        Y_test = Y.iloc[int(0.8*len(Y))+1 : ]
        X_train = X.loc[Y_train.index]
        X_test = X.loc[Y_test.index]
        xgbmodel = xgb.XGBRegressor().fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
        Y_pred= xgbmodel.predict(X_test)
        # score= np.sqrt(mean_squared_error(Y_test, Y_pred)) / Y_pred
        score = root_mean_squared_error(Y_test, Y_pred)
        mean = abs(Y_test).mean()
        CV = score/mean 
        print(proj_id + f'RMSE: {score} \n CV(score/mean):{CV}')
        pd.concat([pd.DataFrame(Y_pred, index= Y_test.index, columns= ['prediction']), Y_test['price'].rename("realized")], axis=1).plot(title= f"{proj_id}", xlabel= 'date', ylabel = 'log_returns(weekly)' )

def predict_next_week(Xs,proj_ids, Y, days_forecast):
    predictions = {}
    for proj_id in proj_ids:
        Y= Y.loc[proj_id]
        X=  Xs.loc[proj_id]
        Y_train= Y.iloc[ : int(0.8*len(Y))]
        Y_test = Y.iloc[int(0.8*len(Y))+1 : ]
        X_train = X.loc[Y_train.index]
        X_test = X.loc[Y_test.index]
        xgbmodel = xgb.XGBRegressor().fit(X_train, Y_train, eval_set = [(X_train, Y_train), (X_test, Y_test)], verbose=False)
        predictions[proj_id] = pd.DataFrame(xgbmodel.predict(X.tail(days_forecast)), index = (X.tail(days_forecast).index + pd.Timedelta(days_forecast, 'days')))
    return predictions

look top cell before change impute to inside loop

In [ ]:
coins = ['BTC','ETH', 'SOL', 'BNB', 'ADA', 'AVAX']
# coins = ['BTC','SOL','BNB','ETH', 'ADA', 'AVAX']
date = pd.to_datetime(dt.datetime.today(), utc =True) - pd.Timedelta(80, 'W')
timeshift = 8
days_forecast= 8
proj_ids= get_proj_ids(coins)

In [ ]:
proj_ids

In [ ]:
FON= get_df_of_OCA(coins, date)

In [ ]:
FON= get_df_of_OCA(coins, date,)
rolled = roll(FON, timeshift)

In [ ]:
# def select_features_on_df(df, y):
#     return tsfresh.select_features(df, y)

# # Extract features
# sample = rolled.partitions[0].compute()
# sample_features = extract_features(sample, column_id='id', column_sort='timestamp', n_jobs=1)
# meta = pd.DataFrame(columns=sample_features.columns)
# for col in meta.columns:
#     meta[col] = meta[col].astype(sample_features[col].dtype)
# FC = rolled.map_partitions(extract_features_on_df, meta=meta)
# FC.index = FC.index.astype(str)

# # Prepare target
# FON = FON.set_index('timestamp')  # Set a single index
# Y = FON[['project_id', 'price']]

# # Shift and transform 'price' for each 'project_id'
# Y['price'] = Y.groupby('project_id')['price'].transform(lambda x: np.log(x.shift(-days_forecast)) - np.log(x))

# # Drop NA values
# Y = Y.dropna()

# # Select features
# Relevant_Features_Target = {}
# for proj_id in FON['project_id'].unique():
#     SF = FC.loc[proj_id].map_partitions(select_features_on_df, Y[Y['project_id'] == proj_id]['price']).persist()
#     Relevant_Features_Target[proj_id] = client.persist(SF)
#     progress(Relevant_Features_Target[proj_id])

In [ ]:
# sample = rolled.partitions[0].compute()
# sample_features = extract_features(sample, column_id='id', column_sort='timestamp', n_jobs=1)
# meta = pd.DataFrame(columns=sample_features.columns)
# for col in meta.columns:
#     meta[col] = meta[col].astype(sample_features[col].dtype)
# FC = rolled.map_partitions(extract_features_on_df, meta=meta)
# future = client.persist(FC)
# progress(future)
# # result = future.result()

In [ ]:
# sample = rolled.partitions[0:4].compute()
# sample_features = extract_features(sample, column_id='id', column_sort='timestamp', n_jobs=20)
# columns = sample_features.columns
# meta = pd.DataFrame(columns=columns)
# for col in meta.columns:
#     meta[col] = meta[col].astype(sample_features[col].dtype)


In [ ]:
Y = FON[['project_id', 'price', 'timestamp']].compute()
Y['price'] = Y.groupby('project_id')['price'].transform(lambda x: np.log(x.shift(-days_forecast)) - np.log(x))
Y = Y.set_index(['project_id','timestamp'])
Y.index.names = [None, None]
Y=Y.dropna()

In [ ]:
extracted_features= extract_features(rolled, column_id='id', column_sort='timestamp', n_jobs=20)
extracted_features= impute(extracted_features)
extracted_features = extracted_features.loc[Y.index]
extracted_features=extracted_features.merge(Y, left_index=True, right_index=True)


In [ ]:
selected_features = select_features(extracted_features.drop('price', axis=1), extracted_features['price'], n_jobs=20)

In [ ]:
train_xgbmodels_on_features(Xs=selected_features, Y=Y, proj_ids=proj_ids, days_forecast=days_forecast)

In [ ]:


# selected_features.index = pd.MultiIndex.from_tuples(selected_features.index.str.split(',').tolist())


In [ ]:
# FC = rolled.map_partitions(extract_features_on_df, meta=meta)
# FC= client.persist(FC)
# progress(FC)

In [ ]:
# def select_features_on_df(df, Y, n_jobs=1):
#     df_imputed = impute(df)
#     y = Y.loc[df_imputed.index]['price'].dropna()
#     df_imputed = df_imputed.loc[y.index]
#     selected_features = select_features(df_imputed, y, n_jobs=1)
#     return selected_features

In [ ]:
# Y = FON[['project_id', 'price', 'timestamp']].compute()
# Y['price'] = Y.groupby('project_id')['price'].transform(lambda x: np.log(x.shift(-days_forecast)) - np.log(x))
# Y = Y.set_index(['project_id','timestamp'])
# Y.index = Y.index.map(str)  # Convert index to string
# meta = pd.DataFrame()
# FC = FC.map_partitions(select_features_on_df, Y=Y, meta = meta)

In [ ]:
# FC=client.compute(FC)

In [ ]:
# FC.result()

In [ ]:
# with ProgressBar():
#     # Compute the shift and log return on the 'price' column before creating Y
#     FON['price_shifted'] = FON.groupby('project_id')['price'].shift(-days_forecast)
#     FON['future_log_return'] = np.log(FON['price_shifted']) - np.log(FON['price'])
#     # # Create Y with the computed columns
#     # Y = FON[['project_id', 'future_log_return', 'timestamp']]
#     # Y= Y.set_index('timestamp')
#     # # Drop NA values
#     # Y = Y.dropna()
#     # Y= Y.compute()

In [ ]:
# FC.index = FC.index.astype(str)

# # Prepare target
# FON = FON.set_index('timestamp')  # Set a single index
# Y = FON[['project_id', 'price']]

# # Shift and transform 'price' for each 'project_id'
# Y['price'] = Y.groupby('project_id')['price'].transform(lambda x: np.log(x.shift(-days_forecast)) - np.log(x))

# # Drop NA values
# Y = Y.dropna()

# # Select features
# Relevant_Features_Target = {}
# for proj_id in FON['project_id'].unique():
#     SF = FC.loc[proj_id].map_partitions(select_features_on_df, Y[Y['project_id'] == proj_id]['price']).persist()
#     Relevant_Features_Target[proj_id] = client.persist(SF)
#     progress(Relevant_Features_Target[proj_id])

In [ ]:
# # Prepare target
# Y = FON[['project_id', 'price', 'timestamp']].set_index([ 'timestamp'])
# Y['price'] = Y['price'].groupby('project_id').shift(-days_forecast)
# Y['price'] = np.log(Y['price']) - np.log(Y['price']).shift(days_forecast)
# Y.dropna(inplace=True)
# def select_features_on_df(df, y):
#     return tsfresh.select_features(df, y)

# # Apply select_features to each partition of FC
# SF = FC.map_partitions(select_features_on_df, Y['price'])

# # Persist computations
# SF = client.persist(SF)

# # Display progress
# progress(SF)

In [ ]:
# Relevant_Features_Target = {}
# for proj_id in FON['project_id'].unique():
#     Y = FON[FON['project_id']==proj_id][['price', 'timestamp']].set_index('timestamp')
#     Y.sort_index(inplace=True)
#     Y['price'] = Y['price'].shift(-days_forecast)
#     Y['price'] = np.log(Y)-np.log(Y).shift(days_forecast)
#     Y.dropna(inplace=True)
#     SF= future.loc[proj_id].map_partitions(select_features, future, Y['price'], n_jobs=1).persist()
#     Relevant_Features_Target[proj_id]= client.persist(SF)
#     progress(Relevant_Features_Target[proj_id])

In [ ]:
# Relevant_Features_Target

In [ ]:
# Selected = select_features_inhouse(FON, result, days_forecast)

In [ ]:
# SET1= ['timestamp', 'project_id','fees', 'price', 'revenue',  'expenses', 'user_dau', 'user_mau', 'contracts_deployed','block_time', 'token_supply_circulating', 'token_trading_volume', 'active_developers']

In [ ]:
# SET2= ['timestamp', 'project_id', 'pf_circulating', 'ps_circulating', 'price', 'contracts_deployed','token_trading_volume', 'token_supply_circulating', 'tokenholders', 'block_time']

In [ ]:
# FON=FON[SET2]

In [ ]:
# Selected = select_features_inhouse(FON, result, days_forecast)

In [ ]:
predict_next_week(selected_features, proj_ids, days_forecast)

RE-WRITE Y AS A COLUMN OF FON SO THAT WEEKLY RETURNS FROM LAST WEEK ARE INCLUDED IN ESTIMATION OF NEXT WEEK